<a href="https://colab.research.google.com/github/turtlejun05/recommendsongbypic/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
keywords = [
    # 감정
    "행복한", "웃는 얼굴", "기쁜 표정", "슬픈", "눈물 흘리는", "외로운", "우울한", "평화로운", "설레는", "감동적인",
    "불안한", "분노한", "놀란 얼굴", "사랑스러운", "따뜻한 마음", "지친 얼굴", "무표정한", "활기찬", "만족스러운",
    "신나는", "감성적인", "무거운",

    # 배경/장소
    "푸른 바다", "해변", "숲속 길", "자연 풍경", "도시 거리", "하늘과 구름", "사막", "눈 덮인 산", "강가", "호수",
    "고요한 마을", "번화한 거리", "벚꽃길", "해넘이 풍경", "등산로", "밤의 도심", "별이 빛나는 하늘", "학교 운동장", "도서관", "지하철역",

    # 날씨
    "맑은 하늘", "흐린 날", "비 오는 거리", "안개 낀 거리", "눈 내리는 풍경", "무지개가 뜬 하늘", "천둥 번개", "태풍이 몰아치는 하늘", "건조한 날씨", "바람 부는 들판",
    "따뜻한 햇살", "쌀쌀한 바람", "이슬 맺힌 풀", "햇빛이 반짝이는 거리", "구름 낀 하늘", "노을이 지는 하늘", "밤에 내리는 비", "서리 낀 창문", "잔잔한 바람", "습한 공기",

    # 시간/계절
    "아침 햇살", "정오의 태양", "해질 무렵", "밤의 정적", "새벽 풍경", "여름 바다", "겨울 산", "봄의 꽃길", "가을 단풍", "해가 뜨는 순간",
    "불꽃놀이 밤", "크리스마스 분위기", "눈 쌓인 거리", "초여름 오후", "늦가을 공원", "비오는 오후", "밤하늘 별빛", "찬란한 오후 햇살", "저녁 노을", "졸린 새벽"
]

emotion_priority = ["슬픈", "설레는", "평화로운", "신나는", "무거운"]

from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
image = Image.open(image_path)

inputs = processor(text=keywords, images=image, return_tensors="pt", padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    probs = outputs.logits_per_image.softmax(dim=1)[0]

emotion_indices = [keywords.index(k) for k in emotion_priority]
emotion_scores = probs[emotion_indices]
top_emotion_idx = emotion_indices[torch.argmax(emotion_scores)]
top_emotion = keywords[top_emotion_idx]

probs[top_emotion_idx] = -1
top_other_indices = torch.topk(probs, k=2).indices.tolist()
top_others = [keywords[i] for i in top_other_indices]

top_keywords = [top_emotion] + top_others
print("\n 이미지 키워드 3개:", top_keywords)

Saving P20200106_105346654_A7A788D2-25EF-459E-88D0-7E4C01759EBC.JPG to P20200106_105346654_A7A788D2-25EF-459E-88D0-7E4C01759EBC.JPG

 이미지 키워드 3개: ['설레는', '가을 단풍', '기쁜 표정']
